# 04_IDAO_Preproc_Test_Interpolate

* поминутная интерполяция значений тестового датасета

In [2]:
import sys
import warnings
warnings.filterwarnings('ignore')

import math
from datetime import datetime, timedelta

import matplotlib.pylab as plt
import pandas as pd
import numpy as np

from scipy import interpolate

In [3]:
#Тренировочные данные
df_train = pd.read_csv('data/train_proc_with_id.csv', index_col='datetime', parse_dates=True)
#Тестовые данные
df_test = pd.read_csv('data/Track 1/test_proc_with_id.csv', index_col='datetime', parse_dates=True)

# Задача 1
submission = pd.read_csv('data/Track 1/submission.csv')

df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [4]:
df_test.head(3)

,id,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
datetime,,,,,,,,
2014-02-01 00:01:45,3927,1,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
2014-02-01 00:22:57,3928,1,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2014-02-01 00:44:08,3929,1,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412


Добавим столбец, переводящий время в абсолютное значение:

In [5]:
df_test = df_test.reset_index()
df_test['time'] = (df_test['datetime'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')

Вычислим индексы с совпадающими метками времени для дальнейшей обработки:

In [6]:
equal_values = []
for i in range(df_test.shape[0] - 1):
    time_prv = list(df_test.loc[i, ['datetime']])
    time_now = list(df_test.loc[i + 1, ['datetime']])
    if time_prv == time_now:
        equal_values.append(i)

Исправим совпадающие временные метки при помощи уравнения равноускоренного движения:

In [8]:
def improve_time_equals(df, equals_list):
    amount = len(equals_list)
    count = 0
    
    for id0 in equals_list:
        id1 = id0 + 1
        id2 = id0 + 2
        
        x0 = float(df.loc[id0, ['x_sim']])
        x1 = float(df.loc[id1, ['x_sim']])
        x2 = float(df.loc[id2, ['x_sim']])

        S02 = x2 - x0

        V0 = float(df.loc[id0, ['Vx_sim']])
        V1 = float(df.loc[id1, ['Vx_sim']])
        V2 = float(df.loc[id2, ['Vx_sim']])

        t0 = float(df.loc[id0, ['time']])
        t2 = float(df.loc[id2, ['time']])
        
        T02 = t2 - t0
        
        a = 2 * (S02 - V0 * T02) / T02**2
        
        A = a / 2
        B = V0
        C = x0 - x1

        discr = B ** 2 - 4 * A * C
        if discr >= 0:
            count += 1
            t1_1 = int((-B - math.sqrt(discr)) / (2 * A))
            t1_2 = int((-B + math.sqrt(discr)) / (2 * A))
            if min(t1_1, t1_2) < 0:
                t1 = max(t1_1, t1_2)
            else:
                t1 = min(t1_1, t1_2)
            time_delta = timedelta(seconds=t1)
            df.loc[id1, ['datetime']] = df.loc[id1, ['datetime']] + time_delta
    return df

In [9]:
df_test_improved = improve_time_equals(df_test, equal_values)

Не все значения удалось исправить. Получим список неисправленных значений:

In [10]:
equal_values = []
for i in range(df_test_improved.shape[0] - 1):
    time_prv = list(df_test_improved.loc[i, ['datetime']])
    time_now = list(df_test_improved.loc[i + 1, ['datetime']])
    if time_prv == time_now:
        equal_values.append(i)

Исправим оставшиеся значения. В качестве метки времени выберем момент времени, равноудалённый от соседних точек.

In [11]:
def improve_time_equals_last(df, equals_list):
    for id0 in equals_list:
        id1 = id0 + 1
        id2 = id0 + 2
        time0 = df.loc[id0, ['time']]
        time2 = df.loc[id0 + 2, ['time']]
        delta = int((time2 - time0)/2)
        time_delta = timedelta(seconds=delta)
        df.loc[id1, ['datetime']] = df.loc[id1, ['datetime']] + time_delta
    return df

In [12]:
df_test_improved = improve_time_equals_last(df_test_improved, equal_values)

Получим список спутников из тестовой выборки:

In [13]:
set_test_satellites_id = set(df_test['sat_id'])

Функции, обеспечивающие интерполяцию значений:

In [14]:
# производит разметку по времени в выборке значений одного спутника
def resample_for_interpolate(df_one_sat):
    df_one_sat['date'] = df_one_sat.index
    df_one_sat['time'] = (df_one_sat['date'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')
    df_one_sat = df_one_sat.resample('min').mean()
    df_one_sat['date'] = df_one_sat.index
    df_one_sat['time'] = (df_one_sat['date'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')
    return df_one_sat

In [15]:
# интерполирует одну колонку датафрейма
def interpolate_col_df(df_one_sat, df_one_sat_resampled, column='x'):
    x = df_one_sat['time']
    y = df_one_sat[column]
    xnew = df_one_sat_resampled['time']
    f = interpolate.interp1d(x, y, fill_value="extrapolate")
    ynew = f(xnew)
    df_one_sat_resampled[column] = ynew
    return df_one_sat_resampled

In [16]:
# интерполирует весь датафрейм
def df_interpolate(df):
    df_result = pd.DataFrame(columns=['sat_id',
                                     'id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
                                     'time', 'date'])
    
    for sat in list(set_test_satellites_id):
        df_one_sat = df.query('sat_id == @sat')
        df_one_sat_resampled = resample_for_interpolate(df_one_sat)

        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'sat_id')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'x_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'y_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'z_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vx_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vy_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vz_sim')
        
        df_result = pd.concat([df_result, df_interpolated])
    
    return df_result

Вернём датасет к исходному виду:

In [17]:
df_test_improved.index = df_test_improved['datetime']
df_test_improved = df_test_improved.drop(['datetime', 'time'], axis=1)

Применим интерполяцию к тестовому датасету:

In [18]:
df_final = df_interpolate(df_test_improved)

Проверим полученный датасет:

In [19]:
df_final.head(3)

,Vx_sim,Vy_sim,Vz_sim,date,id,sat_id,time,x_sim,y_sim,z_sim
2014-02-01 00:01:00,4.307509,-0.730029,0.828370,2014-02-01 00:01:00,3927.0,1.0,1.391213e+09,-13579.030167,-14227.483997,6360.466650
2014-02-01 00:02:00,4.342584,-0.680342,0.804909,2014-02-01 00:02:00,NaN,1.0,1.391213e+09,-13296.178407,-14239.843338,6395.543856
2014-02-01 00:03:00,4.377659,-0.630655,0.781448,2014-02-01 00:03:00,NaN,1.0,1.391213e+09,-13013.326648,-14252.202678,6430.621062


In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12024684 entries, 2014-02-01 00:01:00 to 2014-02-28 22:30:00
Data columns (total 10 columns):
Vx_sim    float64
Vy_sim    float64
Vz_sim    float64
date      datetime64[ns]
id        float64
sat_id    float64
time      float64
x_sim     float64
y_sim     float64
z_sim     float64
dtypes: datetime64[ns](1), float64(9)
memory usage: 1009.2 MB


Сохраним в файл:

In [21]:
df_final.to_csv('data/Track 1/test_final.csv')